In [1]:
import spacy
import textacy
import gensim
import os
import numpy as np
import pandas as pd

In [3]:
review_panel = pd.read_csv('./data/reviews_panel_t8.csv')
review_panel.head()

,Id,review_id,href,overall_rating,Ease_of_Appointment,Promptness,Courteous_Staff,Accurate_Diagnosis,Bedside_Manner,Spends_Time_with_Me,...,Advanced_Technology,Caring_Manner,Pain_Minimized,Satisfaction,date,reviewer,title,content,helpful_vote,crawl_date
0,12949966,23507540,/dentists/Dr_Aarika_Anderson_Elter,5,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,5.0,NaN,2012-11-16,NaN,Great results,Dr. Anderson explained in detail my options fo...,NaN,2016-09-06 20:54:01
1,12949967,27184431,/dentists/Dr_Aamir_Wahab,5,5.0,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,2015-03-15,Brad s.,implant,Had an implant done and it was painless. I cou...,NaN,2016-09-06 20:54:10
2,12949968,26307282,/dentists/Dr_Aanal_Parikh,1,1.0,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,2014-10-12,Bill johnson,Warning ..would not see this dentist,Do not go to this dentist office they scam peo...,NaN,2016-09-06 20:54:13
3,12949969,28904504,/dentists/Dr_Aaron_Aguilar,5,5.0,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,2015-06-11,NaN,NaN,Very thoughtful Dr. Communicates/bedside mann...,NaN,2016-09-06 20:54:14
4,12949970,28380953,/dentists/Dr_Aaron_D_Larsen,4,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,4.0,4.0,5.0,2015-03-31,NaN,Dr larsen great others no,Would continue to see Dr. Larsen however his ...,NaN,2016-09-06 20:54:18


In [5]:
review_panel.shape

(1796204, 23)

In [6]:
review_panel_text = review_panel['title'].fillna('').astype(str) + " " + review_panel['content'].fillna('')
sample_s = review_panel_text.tolist()

## Pre-process via spaCy

In [7]:
nlp = spacy.load('en')

In [8]:
doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion. he his him')

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

Apple apple PROPN NNP nsubj Xxxxx True False
is be VERB VBZ aux xx True True
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True True
buying buy VERB VBG pcomp xxxx True False
U.K. u.k. PROPN NNP compound X.X. False False
startup startup NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False
. . PUNCT . punct . False False
he -PRON- PRON PRP nsubj xx True True
his -PRON- ADJ PRP$ ROOT xxx True True
him -PRON- PRON PRP dobj xxx True True


In [9]:
words_to_keep= set(['he','his','him', 'she','her', 'I','my','me','mine'])
def lemma(x):
    if x.text in words_to_keep: 
        return x.text
    elif x.text not in words_to_keep:
        return x.lemma_
    
    
a_doc = nlp("He said that you didn't buy apples for her. I my me mine")  # feed in string or documents
[lemma(x) for x in a_doc]   # lemmatize words

['-PRON-',
 'say',
 'that',
 '-PRON-',
 'do',
 'not',
 'buy',
 'apple',
 'for',
 'her',
 '.',
 'I',
 'my',
 'me',
 'mine']

In [10]:
# lemmatize and fitler documents; deal with punctuation and typo (2h 30mins)
terms_list=[]
words_to_filter = set(['dr', 'doctor', 'oh'])
words_to_keep = set(['he','his','him','she','her','I','my','me','mine'])
def lemma(x):
    if x.text in words_to_keep: 
        return x.text
    elif x.text not in words_to_keep:
        return x.lemma_

for doc in sample_s:
    doc_cleared = textacy.preprocess_text(doc, lowercase=False, no_punct=True, transliterate=True)
    doc_new = nlp(doc_cleared)
    tokens=[lemma(x) for x in doc_new]  #keep pronoun
    tokens_filtered = [y for y in tokens if y not in words_to_filter]
    terms_list.append(tokens_filtered)

terms_list[:5]

[['great',
  'result',
  'anderson',
  'explain',
  'in',
  'detail',
  'my',
  'option',
  'for',
  'the',
  'repair',
  'on',
  'my',
  'tooth',
  'in',
  'the',
  'end',
  'I',
  'receive',
  'great',
  'result'],
 ['implant',
  'have',
  'an',
  'implant',
  'do',
  'and',
  '-PRON-',
  'be',
  'painless',
  'I',
  'could',
  'not',
  'afford',
  '-PRON-',
  'so',
  'he',
  'put',
  'me',
  'on',
  'a',
  'payment',
  'plan',
  'I',
  'be',
  'pleased',
  'with',
  'the',
  'service',
  'great'],
 ['warning',
  'would',
  'not',
  'see',
  'this',
  'dentist',
  'do',
  'not',
  'go',
  'to',
  'this',
  'dentist',
  'office',
  '-PRON-',
  'scam',
  'people',
  'and',
  'do',
  'not',
  'pay',
  'there',
  'bill',
  '-PRON-',
  'snow',
  'plow',
  'for',
  '-PRON-',
  'and',
  'when',
  'the',
  'season',
  'be',
  'over',
  '-PRON-',
  'would',
  'not',
  'pay',
  '-PRON-',
  'this',
  'be',
  'just',
  'to',
  'let',
  '-PRON-',
  'know',
  'what',
  'type',
  'of',
  'dentist',

In [20]:
# save the corpus as a txt file (2 min)
with open("./data/terms_list_sample.txt","w") as thefile:
    for item in terms_list:
      thefile.write("%s\n" % item)

## Build model via word2vec

In [12]:
# read in corpus
sts_terms_list = gensim.models.word2vec.LineSentence('terms_list_sample.txt')

In [30]:
# train word2vec on the corpus (10 mins)
model = gensim.models.Word2Vec(sentences=terms_list, size=100, window=7, min_count=5, workers=4)
# to save a trained model
model.save("./data/model_doc_comments1")

In [2]:
# load the trained model
model = gensim.models.Word2Vec.load("./data/model_doc_comments1")

In [31]:
# To convert documents to vectors (10 min)
from gensim import corpora
dictionary = corpora.Dictionary(terms_list)
# store the dictionary, for future reference
dictionary.save('./data/try_dictionary.dict')
#print(dictionary)

In [32]:
# To see the mapping between words and their ids
print(dictionary.token2id)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [3]:
model.wv['I']  # numpy vector of a word

array([ 0.8985168 ,  0.21133737, -1.0174987 ,  0.23950161,  0.6535615 ,
       -2.1597543 , -0.96719617, -1.2723322 ,  0.6952619 ,  0.52317786,
        3.8957522 , -4.2797956 , -3.5484123 ,  2.9083505 , -2.0390408 ,
        1.4697789 ,  2.7010856 , -2.316967  ,  1.357116  ,  2.9058228 ,
       -1.5485703 , -2.5820878 ,  1.5781274 , -4.7464647 ,  2.5480347 ,
       -1.0955589 , -4.0962763 ,  2.5019717 , -1.9741727 ,  2.3681483 ,
        2.2892256 , -1.7966137 ,  2.341933  ,  1.9638792 ,  1.0589308 ,
       -3.6643832 ,  0.8503319 ,  2.3559475 ,  0.8861464 ,  2.4579482 ,
        0.09558363, -1.3544432 , -1.3810408 ,  3.4680386 , -2.2576418 ,
       -1.45918   ,  1.1770133 ,  0.47717357,  0.82000923,  1.227406  ,
       -3.9137058 , -0.06107468, -0.597307  , -2.9509046 , -1.9264892 ,
       -0.3778871 , -1.1883132 ,  2.1589031 , -0.23421006,  0.4014843 ,
       -1.801502  , -2.567079  , -1.50274   , -1.3262734 ,  1.2228625 ,
       -3.5384681 ,  2.4423347 ,  1.5887536 ,  1.7367889 ,  0.56

In [5]:
model['I']

/Users/xiaowan/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.8985168 ,  0.21133737, -1.0174987 ,  0.23950161,  0.6535615 ,
       -2.1597543 , -0.96719617, -1.2723322 ,  0.6952619 ,  0.52317786,
        3.8957522 , -4.2797956 , -3.5484123 ,  2.9083505 , -2.0390408 ,
        1.4697789 ,  2.7010856 , -2.316967  ,  1.357116  ,  2.9058228 ,
       -1.5485703 , -2.5820878 ,  1.5781274 , -4.7464647 ,  2.5480347 ,
       -1.0955589 , -4.0962763 ,  2.5019717 , -1.9741727 ,  2.3681483 ,
        2.2892256 , -1.7966137 ,  2.341933  ,  1.9638792 ,  1.0589308 ,
       -3.6643832 ,  0.8503319 ,  2.3559475 ,  0.8861464 ,  2.4579482 ,
        0.09558363, -1.3544432 , -1.3810408 ,  3.4680386 , -2.2576418 ,
       -1.45918   ,  1.1770133 ,  0.47717357,  0.82000923,  1.227406  ,
       -3.9137058 , -0.06107468, -0.597307  , -2.9509046 , -1.9264892 ,
       -0.3778871 , -1.1883132 ,  2.1589031 , -0.23421006,  0.4014843 ,
       -1.801502  , -2.567079  , -1.50274   , -1.3262734 ,  1.2228625 ,
       -3.5384681 ,  2.4423347 ,  1.5887536 ,  1.7367889 ,  0.56